<a href="https://colab.research.google.com/github/lucianomcsilva/RIWRS/blob/main/%5BRIWRS%5D_3_books_scrapy_via_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapy



In [1]:
# Instalando bibliotecas
!pip install scrapy
!pip install crochet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 4.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# text cleaning
import re
import scrapy
import logging
from scrapy.crawler import CrawlerRunner
# Reactor restart
from crochet import setup, wait_for
setup()

In [3]:
class SpiderBooksSpider(scrapy.Spider):
    name = 'spider_books'
    allowed_domains = ['books.toscrape.com']
    start_urls = ['http://books.toscrape.com/']
    
    custom_settings = {
        'LOG_LEVEL': 'DEBUG',
        'FEED_FORMAT':'csv',                            # Used for pipeline
        'FEED_URI': 'books.csv',                       # Used for pipeline     
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }

    
    def parse(self, response):
        for book in response.css("ol.row").css('li'): 
            link = response.urljoin(book.css("h3").css("a::attr(href)").get())
            print(link)
            yield scrapy.Request(link, callback=self.parse_details)
        
        next_page = response.css("li.next a::attr(href)").get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

    def RatingToInt(self, strRating: str) -> int:
            if strRating == 'One':
                return 1
            elif strRating == 'Two':
                return 2
            elif strRating == 'Three':
                return 3
            elif strRating == 'Four':
                return 4
            elif strRating == 'Five':
                return 5
            else:
                return 0   # Valor padrão
    
    def parse_details(self, response):
        name = response.css("h1::text").get()
        price = response.css("p.price_color::text").get()[1:]
        rating = response.css('.product_main').css('p.star-rating::attr(class)').get()[12:]
        upc    = response.css("td::text").get()

        yield {
            "name": name,
            "preço": price,
            'upc': upc,
            'rating': self.RatingToInt(rating)
        }        


Inicia o crawler

In [4]:
crawler = CrawlerRunner()
crawler.crawl(SpiderBooksSpider)


It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
